In [1]:
pwd

'/content'

In [2]:
cd ./drive/MyDrive/Data_analysis

/content/drive/MyDrive/Data_analysis


In [3]:
ls -l

total 75
-rw-------  1 root root 71013 Aug 13 07:00 '멕시코풍 프랜차이즈 chipotle의 주문데이터 분석하기.ipynb'
-rw-------  1 root root  1039 Aug 19 00:56 '술데이터 분석 응용.ipynb'
drwx------ 11 root root  4096 Aug 11 06:33  python-data-analysis/


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

file_path = "./python-data-analysis/data/drinks.csv"
drinks = pd.read_csv(file_path)
drinks['continent'] = drinks['continent'].fillna('OT')
drinks

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
0,Afghanistan,0,0,0,0.0,AS
1,Albania,89,132,54,4.9,EU
2,Algeria,25,0,14,0.7,AF
3,Andorra,245,138,312,12.4,EU
4,Angola,217,57,45,5.9,AF
...,...,...,...,...,...,...
188,Venezuela,333,100,3,7.7,SA
189,Vietnam,111,2,1,2.0,AS
190,Yemen,6,0,0,0.1,AS
191,Zambia,32,19,4,2.5,AF


In [8]:
# 1. 대륙별 평균 wine_servings를 탐색 
result = drinks.groupby('continent').wine_servings.agg(['mean'])
result.head()
"""
group by연산  결과를 반환하면 반환 데이터 타입은 dataframe 타입이 아니다.
따라서 to_frame이라는 함수를 사용하여  dataframe타입으로 변환 한 뒤 
다시 dataframe 의  index라는 함수로 설정하는 과정이 필요하다, 그리고 이 과정으로 생성된  dataframe을 원래의 dataframe에 합쳐주어야한다.
이를 위해 merge라는 함수를 또 적용해야하지만 transform 함수를 사용하면 이 모든과정을 단 한줄의 코드로 수행가능 
"""

,mean
continent,
AF,16.264151
AS,9.068182
EU,142.222222
OC,35.625000
OT,24.521739


In [31]:
result = drinks.groupby('continent').mean()['wine_servings']
df = result.to_frame().reset_index()
df = df.rename(columns={'wine_servings': 'wine_servings_cont_avg'})
drinks = pd.merge(drinks,df,on='continent',how='outer')

#위와 같은 방법의 코드 
drinks['wine_servings_cont_avg'] = drinks.groupby('continent')['wine_servings'].transform(np.mean)


In [32]:
drinks[['country','continent','wine_servings_cont_avg']].sample(5).head()

,country,continent,wine_servings_cont_avg
15,Kazakhstan,AS,9.068182
74,Norway,EU,142.222222
189,Solomon Islands,OC,35.625000
191,Tuvalu,OC,35.625000
96,Cabo Verde,AF,16.264151


In [9]:
#2 국가별 모든 servings의 합을 계산한 total_servings라는 피처를 생성한다. 
drinks['total_servings'] = drinks['beer_servings'] + drinks['wine_servings'] + drinks['spirit_servings']
drinks

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent,total_servings
0,Afghanistan,0,0,0,0.0,AS,0
1,Albania,89,132,54,4.9,EU,275
2,Algeria,25,0,14,0.7,AF,39
3,Andorra,245,138,312,12.4,EU,695
4,Angola,217,57,45,5.9,AF,319
...,...,...,...,...,...,...,...
188,Venezuela,333,100,3,7.7,SA,436
189,Vietnam,111,2,1,2.0,AS,114
190,Yemen,6,0,0,0.1,AS,6
191,Zambia,32,19,4,2.5,AF,55


In [33]:
drinks[['country','beer_servings','wine_servings','spirit_servings','total_servings']].sample(5).head()

,country,beer_servings,wine_servings,spirit_servings,total_servings
98,Central African Republic,17,1,2,20
34,Syria,5,16,35,56
157,Mexico,238,5,68,311
117,Madagascar,26,4,15,45
70,Malta,149,120,100,369


In [14]:
#3전체 평균보다 적은 알코올을 마시는 대륙중, spirit을 가장 많이 마시는 국가를 찾아본다.
#1) 전체 평균 보다 적은 알코올을 마시는 대륙을 먼저 찾는다.
total_mean = drinks.total_litres_of_pure_alcohol.mean()
continent_mean = drinks.groupby('continent')['total_litres_of_pure_alcohol'].mean()
continent_under_mean = continent_mean[continent_mean <= total_mean].index.tolist()
continent_under_mean 


['AF', 'AS', 'OC']

In [16]:
#2) spirit을 가장 많이 마시는 국가를 찾아본다.
df_continent_under_mean = drinks.loc[drinks.continent.isin(continent_under_mean)]
df_continent_under_mean


,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent,total_servings
0,Afghanistan,0,0,0,0.0,AS,0
2,Algeria,25,0,14,0.7,AF,39
4,Angola,217,57,45,5.9,AF,319
8,Australia,261,72,212,10.4,OC,545
12,Bahrain,42,63,7,2.0,AS,112
...,...,...,...,...,...,...,...
187,Vanuatu,21,18,11,0.9,OC,50
189,Vietnam,111,2,1,2.0,AS,114
190,Yemen,6,0,0,0.1,AS,6
191,Zambia,32,19,4,2.5,AF,55


In [18]:
most_spirit_under_mean = df_continent_under_mean.loc[df_continent_under_mean['spirit_servings'].idxmax()]
most_spirit_under_mean['country']

'Russian Federation'

In [39]:
#4 술소비량 대비 알코올 비율을 구해본다
drinks['alcohol_rate'] = drinks['total_litres_of_pure_alcohol'] / drinks['total_servings']
drinks['alcohol_rate'] = drinks['alcohol_rate'].fillna(0)
drinks


drinks['alcohol_rate_rank'] = drinks['alcohol_rate'].rank(ascending = False)
drinks['alcohol_rate_rank'] = drinks['alcohol_rate_rank'].apply(np.floor)
drinks.loc[drinks['country'] == 'South Korea'].alcohol_rate_rank

29    15.0
Name: alcohol_rate_rank, dtype: float64

In [29]:
#5 대륙별 술 소비량 대비 알코올 비율을 구한다. 
continent_with_rank = drinks.groupby('continent')['alcohol_rate'].mean()
continent_with_rank

continent
AF    0.049741
AS    0.016359
EU    0.017527
OC    0.016895
OT    0.017854
SA    0.017868
Name: alcohol_rate, dtype: float64

In [42]:
continent_sum = drinks.groupby('continent').sum()
continent_sum['alcohol_rate_continent'] = continent_sum['total_litres_of_pure_alcohol'] / continent_sum['total_servings']

continent_sum = continent_sum.reset_index()
continent_sum = continent_sum[['continent','alcohol_rate_continent']]

drinks= pd.merge(drinks,continent_sum,on='continent',how='outer')

In [43]:
drinks[['country','continent','alcohol_rate_continent']].sample(5).head()

,country,continent,alcohol_rate_continent
127,Rwanda,AF,0.031970
5,Cambodia,AS,0.020293
67,Latvia,EU,0.018392
99,Chad,AF,0.031970
177,Australia,OC,0.018401
